<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/kursPawe%C5%82Krakowiak/machine-learning-bootcamp/unsupervised/03_association_rules/01_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### scikit-learn
Strona biblioteki: [https://scikit-learn.org](https://scikit-learn.org)  

Dokumentacja/User Guide: [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)

Podstawowa biblioteka do uczenia maszynowego w języku Python.

Aby zainstalować bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install scikit-learn
```
Aby zaktualizować do najnowszej wersji bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install --upgrade scikit-learn
```
Kurs stworzony w oparciu o wersję `0.22.1`

### Spis treści:
1. [Import bibliotek](#0)
2. [Wygenerowanie danych](#1)
3. [Przygotowanie danych](#2)
4. [Algorytm Apriori](#3)




### <a name='0'></a> Import bibliotek

In [1]:
print("############################################################")
print("Etap12. Odcinek: Reguły asocjacyjne - algorytm Apriori - ćwiczenie 1")

# Wykład:

# Niech:
# P - zbiór wszystkich dostępnych produktów
# T - baza transakcji
#  A i B - produkty
# n - liczba produktów
# A->B reguła

# Wsparcie produktu:
# support(A, T) = (liczba transakcji w których jest A) / (liczba wszystkich transakcji z bazy T)

# Wsparcie reguły:
# support(A->B, T) = (liczba transakcji w których jest A i B) / (liczba wszystkich transakcji z bazy T)
# w skrócie supp(A->B)

# Zaufanie (wiarygodność) reguły. Mówi na ile ta reguła się sprawdza mając jej poprzednik:
# confidence(A->B, T) = (liczba transakcji w których jest A i B) / (liczba transakcji w których jest A)
# w skrócie conf(A->B)
# Wśród klientów kupujących A taka część kupuje też B

# Ważność reguły:
# lift(A->B) = conf(A->B) / supp(B)

# Algorytm Apriori:
# 1) Określenie minimalnego poziomu wsparcia i wiarygodności reguły
# 2) Pozostawienie reguł mających nie mniejszy poziom wspacia niż minimalny poziom
# 3) Pozostawienie reguł mających wiarygodność nie miniejszą niż minimalny poziom
# 4) Posortowanie reguł według malejącej wartości lift

import pandas as pd
import numpy as np

############################################################
Etap12. Odcinek: Reguły asocjacyjne - algorytm Apriori - ćwiczenie 1


### <a name='1'></a> Wygenerowanie danych

In [2]:
data = {'produkty': ['chleb jajka mleko', 'mleko ser', 'chleb masło ser', 'chleb jajka']}

transactions = pd.DataFrame(data=data, index=[1, 2, 3, 4])
transactions

,produkty
1,chleb jajka mleko
2,mleko ser
3,chleb masło ser
4,chleb jajka


### <a name='2'></a> Przygotowanie danych

In [3]:
# rozwinięcie kolumny do obiektu DataFrame
expand = transactions['produkty'].str.split(expand=True)
expand

,0,1,2
1,chleb,jajka,mleko
2,mleko,ser,None
3,chleb,masło,ser
4,chleb,jajka,None


In [4]:
# wydobycie nazw wszystkich produktów
products = []
for col in expand.columns:
    for product in expand[col].unique():
        if product is not None and product not in products:
            products.append(product)

products.sort()
print(products)

['chleb', 'jajka', 'masło', 'mleko', 'ser']


In [5]:
# wierszy tyle co transakcji czyli len(transactions)
# kolumn tyle co produktów czyli len(products)
# macierz samych zer
transactions_encoded = np.zeros((len(transactions), len(products)), dtype='int8')
transactions_encoded

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int8)

In [6]:
# kodowanie 0-1
for row in zip(range(len(transactions)), transactions_encoded, expand.values):
    for idx, product in enumerate(products):
        if product in row[2]:
            transactions_encoded[row[0], idx] = 1

transactions_encoded

array([[1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [1, 1, 0, 0, 0]], dtype=int8)

In [7]:
transactions_encoded_df = pd.DataFrame(transactions_encoded, columns=products)
transactions_encoded_df

,chleb,jajka,masło,mleko,ser
0,1,1,0,1,0
1,0,0,0,1,1
2,1,0,1,0,1
3,1,1,0,0,0


### <a name='3'></a> Algorytm Apriori

In [8]:
from mlxtend.frequent_patterns import apriori, association_rules

# Liczy support
# Ustawił min_support=0.0 żeby zobaczyć wszystkie możliwe kombinacje
# use_colnames=True używa kolumn jako nazwy produktów
supports = apriori(transactions_encoded_df, min_support=0.0, use_colnames=True)
supports

,support,itemsets
0,0.75,(chleb)
1,0.50,(jajka)
2,0.25,(masło)
3,0.50,(mleko)
4,0.50,(ser)
5,0.50,"(jajka, chleb)"
6,0.25,"(masło, chleb)"
7,0.25,"(mleko, chleb)"
8,0.25,"(ser, chleb)"
9,0.00,"(masło, jajka)"


In [9]:
# min_support=0.3
supports = apriori(transactions_encoded_df, min_support=0.3, use_colnames=True)
supports

,support,itemsets
0,0.75,(chleb)
1,0.50,(jajka)
2,0.50,(mleko)
3,0.50,(ser)
4,0.50,"(jajka, chleb)"


In [10]:
# supports to co wyszło z funkcji apriori
# antecedent to poprzednik, a consequent to następnik
rules = association_rules(supports, metric='confidence', min_threshold=0.65)
# bierze tylko niektóre kolumny bo funkcja association_rules zwraca dużo informacji
rules = rules.iloc[:, [0, 1, 4, 5, 6]]
rules

,antecedents,consequents,support,confidence,lift
0,(jajka),(chleb),0.5,1.000000,1.333333
1,(chleb),(jajka),0.5,0.666667,1.333333


In [11]:
# sortowanie względem lift
rules.sort_values(by='lift', ascending=False)

,antecedents,consequents,support,confidence,lift
0,(jajka),(chleb),0.5,1.000000,1.333333
1,(chleb),(jajka),0.5,0.666667,1.333333
